In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [2]:
import scipy.ndimage as ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import build_montages
import matplotlib.pyplot as plt
import cv2
from keras.utils import np_utils
from imutils.contours import sort_contours
import imutils

In [3]:
from google.colab.patches import cv2_imshow

In [4]:
# change num_classes value depends on corresponding dataset
num_classes = 75
img_size = 60

In [5]:
dataset=pd.read_csv("./datasets/newdataset-60-75-x2.csv").astype('float32')

In [ ]:
dataset=pd.read_csv("./datasets/newdataset-60-62-x2.csv").astype('float32')

In [7]:
newdataset=np.array(dataset)

In [8]:

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(newdataset, newdataset[:,0]):
  strat_train_set = newdataset[train_index]
  strat_val_test_set = newdataset[test_index]

In [9]:

split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for test_index, val_index in split2.split(strat_val_test_set, strat_val_test_set[:,0]):
  strat_test_set = strat_val_test_set[test_index]
  strat_val_set = strat_val_test_set[val_index]

In [10]:

X = np.array(strat_train_set[:,1:])/255.
X_train=X.reshape(X.shape[0],img_size,img_size,1)
y = strat_train_set[:,0]

labels = keras.utils.to_categorical(y, num_classes)

X_val=  np.array(strat_val_set[:,1:])/255
X_val=X_val.reshape(X_val.shape[0],img_size,img_size,1)
y_val = strat_val_set[:,0]
labels_val=keras.utils.to_categorical(y_val, num_classes)

X_test=  np.array(strat_test_set[:,1:])/255
X_test=X_test.reshape(X_test.shape[0],img_size,img_size,1)
y_test = strat_test_set[:,0]
labels_test=keras.utils.to_categorical(y_test, num_classes)

In [11]:
model = keras.models.Sequential()
# model.add(keras.layers.Reshape((img_size,img_size,1), input_shape=(784,)))
model.add(keras.layers.Conv2D(filters=12, kernel_size=(5,5), strides=2, activation='relu', input_shape=(img_size,img_size,1)))
model.add(keras.layers.Dropout(.5))

model.add(keras.layers.Conv2D(filters=18, kernel_size=(3,3) , strides=2, activation='relu'))
model.add(keras.layers.Dropout(.5))

model.add(keras.layers.Conv2D(filters=24, kernel_size=(2,2), activation='relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=150, activation='relu'))
model.add(keras.layers.Dense(units=num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 12)        312       
                                                                 
 dropout (Dropout)           (None, 28, 28, 12)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 18)        1962      
                                                                 
 dropout_1 (Dropout)         (None, 13, 13, 18)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 24)        1752      
                                                                 
 flatten (Flatten)           (None, 3456)              0         
                                                                 
 dense (Dense)               (None, 150)               5

In [13]:
checkpoint_filepath ="./models/bestmodel1-3.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, labels, validation_data=(X_val, labels_val), epochs=100, batch_size=200, verbose=2,callbacks=[model_checkpoint_callback])


Epoch 1/100

Epoch 1: val_accuracy improved from -inf to 0.54545, saving model to /content/drive/MyDrive/checkpoint-model3/bestmodel1-3.hdf5
17/17 - 4s - loss: 3.6351 - accuracy: 0.1724 - val_loss: 1.8686 - val_accuracy: 0.5455 - 4s/epoch - 228ms/step
Epoch 2/100

Epoch 2: val_accuracy improved from 0.54545 to 0.76479, saving model to /content/drive/MyDrive/checkpoint-model3/bestmodel1-3.hdf5
17/17 - 3s - loss: 1.3157 - accuracy: 0.6199 - val_loss: 0.7059 - val_accuracy: 0.7648 - 3s/epoch - 149ms/step
Epoch 3/100

Epoch 3: val_accuracy improved from 0.76479 to 0.83983, saving model to /content/drive/MyDrive/checkpoint-model3/bestmodel1-3.hdf5
17/17 - 3s - loss: 0.7809 - accuracy: 0.7623 - val_loss: 0.5365 - val_accuracy: 0.8398 - 3s/epoch - 151ms/step
Epoch 4/100

Epoch 4: val_accuracy improved from 0.83983 to 0.84848, saving model to /content/drive/MyDrive/checkpoint-model3/bestmodel1-3.hdf5
17/17 - 3s - loss: 0.5760 - accuracy: 0.8174 - val_loss: 0.4789 - val_accuracy: 0.8485 - 3s/ep

In [16]:
preds = model.evaluate(X_test, labels_test)

22/22 [==============================] - 0s 9ms/step - loss: 0.0751 - accuracy: 0.9784
